In [1]:
import numpy as np
import copy
from collections.abc import Iterable
from dragon.search_space.base_variables import Variable, IntVar, FloatVar, CatVar, ArrayVar
from dragon.search_operators.base_neighborhoods import VarNeighborhood, IntInterval, FloatInterval, CatInterval

In [29]:
class ExclusiveBlock(Variable):
    """ExclusiveBlock(Variable)

    `ExclusiveBlock` defines `var` which will repeat multiple times a `var` while making sure each occurrence is unique.

    Parameters
    ----------
    label : str
        Name of the variable.
    value : Variable
        :ref:`var` that will be repeated
    repeat : int
        Number of repeats.

    Examples
    --------
    """
    def __init__(self, label, value, repeat, **kwargs):
        self.value = value
        super(ExclusiveBlock, self).__init__(label, **kwargs)

        assert isinstance(
            value, Variable
        ), f"""
        Value must inherit from :ref:`var`, got {value}
        """
        assert (
            isinstance(repeat, int) and repeat > 0
        ), f"""
        `repeat` must be a strictly positive int, got {repeat}.
        """

        if (repeat > 1):
            assert (
                not value.isconstant()
            ), f"""
            Cannot repeat a constant 'value' more than one time while ensuring unicity of the 'value'.
            """

        if isinstance(value, CatVar):
            assert(
                len(value.features) >= repeat
            )  , f"""
            There are not enough distinct values for the 'value' CatVar so as to create 'repeat' unique occurrences.
            """

        self.repeat = repeat
    
    def random(self, size=1):
        res = []

        for _ in range(size):
            block = []
            for _ in range(self.repeat):
                if isinstance(self.value, list) and (len(self.value) > 0):
                    valueInstance = []
                    for i in range(len(self.value)):
                        temp = self.value[i].random()
                        while temp in block[:][i]:
                            temp = self.value[i].random()
                        valueInstance.append(temp)
                    block.append(valueInstance)
                else:
                    temp = self.value.random()
                    while temp in block:
                        temp = self.value.random()
                    block.append(temp)
            res.append(block)

        if (size == 1):
            return res[0]
        else:
            return res
    
    def isconstant(self):
        return self.value.isconstant()

    def __repr__(self):
            values_reprs = ""
            if isinstance(self.value, Iterable) and (len(self.value) > 0):
                for v in self.value:
                    values_reprs += v.__repr__() + ","

                return super(ExclusiveBlock, self).__repr__() + f"[{values_reprs}])"
            else:
                return super(ExclusiveBlock, self).__repr__() + f"{self.value.__repr__()}"


class DynamicExclusiveBlock(ExclusiveBlock):
    def __init__(self, label, value, max_repeat, min_repeat=1, **kwargs):
        self.value = value

        assert (
            isinstance(max_repeat, int) and max_repeat > 0
        ), f"""
        `max_repeat` must be a strictly positive int, got {max_repeat}.
        """

        assert (
            isinstance(min_repeat, int) and min_repeat > 0
        ), f"""
        `min_repeat` must be a strictly positive int, got {min_repeat}.
        """

        assert (
            min_repeat <= max_repeat
        ), f"""
        'min_repeat' must be less than or equal to 'max_repeat', got {min_repeat} <= {max_repeat}
        """

        self.min_repeat = min_repeat        
        super(DynamicExclusiveBlock, self).__init__(label, value, max_repeat, **kwargs)


    def random(self, size=1, n_repeat=None):
        """random(size=1)

        Parameters
        ----------
        size : int, default=None
            Number of draws.
        n_repeat : max size of randomly generated block

        Returns
        -------
        out: float or list[float]
            Return a list composed of the results from the `var` `random()`
            method, repeated `repeat` times with unique occurrences.
            If size > 1, return a list of list.
        """
        
        res = []

        for _ in range(size):
            block = []
            if n_repeat is None:
                n_repeat = np.random.randint(self.min_repeat, self.repeat)
            block = ExclusiveBlock("block", self.value, n_repeat).random()
            res.append(block)

        if (size == 1):
            return res[0]
        else:
            return res

    def isconstant(self):
        """isconstant()

        Returns
        -------
        out: False
            Return False, an exclusive dynamic block cannot be constant. (It is a binary)

        """
        return False


class GamCoef(Variable):
    """
    Coefficient to be included in a GAM
    Special format for the "temp_lissx" coefficient: it has a smoothing factor as well
    Special format for the "DayType" and "offset" coefficients: they have a list of types

    Parameters
    ----------
    feature_list     : List of all features that can be included in a generalized additive model
    categorical_list : List of categorical features among those in feature_list
    spline_list      : List of splines that can be used in a generalized additive model, including "linear" to represent variables entering parametrically
    k_min            : Minimum degree of freedom for a spline
    k_max            : Maximum degree of freedom for a spline
    k_interval       : Degree of freedom variation amplitude for a mutation (new_k = current_k ± k_interval with new_k automatically staying within its bounds)
    bivariate        : Boolean to specify whether bivariate effects must be considered or not
    
    Specific to EDF R39
    alpha_min      : Minimum smoothing coefficient for the temperature
    alpha_max      : Maximum smoothing coefficient for the temperature
    alpha_interval : Smoothing coefficient variation amplitude for a mutation (new_alpha = current_alpha ± alpha_interval with new_alpha automatically staying within its bounds)
    daytype_list   : List of daytypes to be calculated
    offset_list    : List of offsets to be calculated
    """
    def __init__(self, label, feature_list, categorical_list, spline_list, k_min=3, k_max=30, bivariate=False, alpha_min=0.9, alpha_max=0.99, daytype_list=None, offset_list=None, **kwargs):
        self.feature_list = feature_list
        self.spline_list = spline_list
        self.categorical_list = categorical_list
        self.bivariate = bivariate
        
        #Note: k has no meaning when the corresponding spline is linear
        self.k_min = k_min
        self.k_max = k_max

        #Specific to EDF R39
        self.alpha_min = alpha_min
        self.alpha_max = alpha_max
        self.daytype_list = daytype_list
        self.offset_list = offset_list
        if (daytype_list != None):
            self.daytype_dynExcBl = DynamicExclusiveBlock("daytype_dynExcBl", CatVar("daytype_catvar", self.daytype_list), min_repeat=2, max_repeat=len(daytype_list))
        if (offset_list != None):
            self.offset_dynExcBl = DynamicExclusiveBlock("offset_dynExcBl", CatVar("offset_catvar", self.offset_list), min_repeat=2, max_repeat=len(offset_list))

        super(GamCoef, self).__init__(label, **kwargs)

    def random(self, size=1):        
        res = []

        for _ in range(size):
            new_coef = []
            if (self.bivariate and np.random.binomial(1, 0.3) == 1):
                new_coef.append(np.random.choice(["s", "te", "ti", "t2"]))
                new_coef.append(np.random.choice(self.feature_list))

                #Avoiding having the same bivariate effect twice
                if (new_coef[1] in self.categorical_list):
                    selectable_feat_2 = [x for x in self.feature_list if not x in self.categorical_list]
                    if (selectable_feat_2 == []):
                        continue
                    new_coef.insert(1, np.random.choice(selectable_feat_2))
                else:
                    selectable_feat_2 = [x for x in self.feature_list if x != new_coef[1]]
                    if (selectable_feat_2 == []):
                        continue
                    new_coef.append(np.random.choice(selectable_feat_2))

                #Generating a smoothing coefficient for the first feature if necessary
                #as the first feature is necessarily non-categorical
                if (new_coef[1][:-1] == "temp_liss"):
                    new_coef.insert(2, np.random.uniform(self.alpha_min, self.alpha_max))

            else:
                #Ensuring that the new feature has not already been selected in the GAM
                new_coef.append(np.random.choice(self.feature_list))

            #Generating a smoothing coefficient if necessary
            if (new_coef[-1][:-1] == "temp_liss"):
                new_coef.append(np.random.uniform(self.alpha_min, self.alpha_max))
            #Generating a list of types if necessary
            elif (new_coef[-1] == "offset"):
                new_coef.append(self.offset_dynExcBl.random())
            elif (new_coef[-1] == "DayType"):
                new_coef.append(self.daytype_dynExcBl.random())                      

            if (new_coef[0] in self.categorical_list):
                new_coef.append("linear")
                new_coef.append(0)
            else:
                new_coef.append(np.random.choice(self.spline_list))
                new_coef.append(np.random.randint(self.k_min, self.k_max+1))
                #These splines only work in R with k=4 at least
                if (new_coef[-2] in ['ds', 'cc', 'ps']):
                    new_coef[-1] = max(new_coef[-1], 4)
                #This spline only works in R with k=5 at least
                elif (new_coef[-2] == 'bs'):
                    new_coef[-1] = max(new_coef[-1], 5)
            
            res.append(new_coef)
        
        if (size==1):
            return res[0]
        else:
            return res
    
    def isconstant(self):                
        return False
    

class GAM(Variable):
    """
    List of coefficients to be included in a GAM
    Special format for the "temp_lissx" coefficient: it has a smoothing factor as well
    Special format for the "DayType" and "offset" coefficients: they have a list of types

    Parameters
    ----------
    feature_list     : List of features that can be included in a generalized additive model
    categorical_list : List of categorical features among those in feature_list
    spline_list      : List of splines that can be used in a generalized additive model, including "linear" to represent variables entering parametrically
    k_min            : Minimum degree of freedom for a spline
    k_max            : Maximum degree of freedom for a spline
    bivariate        : Boolean to specify whether bivariate effects must be considered or not
    
    Specific to EDF R39
    alpha_min      : Minimum smoothing coefficient for the temperature
    alpha_max      : Maximum smoothing coefficient for the temperature
    daytype_list   : List of daytypes to be calculated
    offset_list    : List of offsets to be calculated
    """
    def __init__(self, label, feature_list, categorical_list, spline_list, k_min=3, k_max=30, bivariate=False, alpha_min=0.9, alpha_max=0.99, daytype_list=None, offset_list=None, **kwargs):
        self.feature_list = feature_list
        self.spline_list = spline_list
        self.categorical_list = categorical_list
        self.bivariate = bivariate

        #Note: k has no meaning when the corresponding spline is linear
        self.k_min = k_min
        self.k_max = k_max

        #Specific to EDF R39
        self.alpha_min = alpha_min
        self.alpha_max = alpha_max
        self.daytype_list = daytype_list
        self.offset_list = offset_list
        if (daytype_list != None):
            self.daytype_dynExcBl = DynamicExclusiveBlock("daytype_dynExcBl", CatVar("daytype_catvar", self.daytype_list), min_repeat=2, max_repeat=len(daytype_list))
        if (offset_list != None):
            self.offset_dynExcBl = DynamicExclusiveBlock("offset_dynExcBl", CatVar("offset_catvar", self.offset_list), min_repeat=2, max_repeat=len(offset_list))

        self.maxVar = len(feature_list) #Maximum number of features includable in the GAM

        super(GAM, self).__init__(label, **kwargs)
    
    def random(self, size=1):
        res = []
        for _ in range(size):

            #Number of features in the generated GAM
            Nvar = np.random.randint(1, 2*self.maxVar+1)   ###### MAX NUMBER OF VAR IN BIVARIATE CASE ? ######
            gam = []
            selected_features = []
            selected_bivar = []

            for k in range(Nvar):
                new_coef = []

                if (self.bivariate and np.random.binomial(1, 0.3) == 1):
                    new_coef.append(np.random.choice(["s", "te", "ti", "t2"]))
                    new_coef.append(np.random.choice(self.feature_list))

                    selected_feat_2 = [new_coef[1]]
                    for i in range(len(selected_bivar)):
                        if (new_coef[1] == selected_bivar[i][0]):
                            selected_feat_2.append(selected_bivar[i][1])
                        elif (new_coef[1] == selected_bivar[i][1]):
                            selected_feat_2.append(selected_bivar[i][0])
                    
                    #Avoiding having the same bivariate effect twice
                    if new_coef[1] in self.categorical_list:
                        selectable_feat_2 = [x for x in self.feature_list if not x in selected_feat_2 and not x in self.categorical_list]
                        if (selectable_feat_2 == []):
                            continue
                        new_coef.insert(1, np.random.choice(selectable_feat_2))
                    else:
                        selectable_feat_2 = [x for x in self.feature_list if not x in selected_feat_2]
                        if (selectable_feat_2 == []):
                            continue
                        new_coef.append(np.random.choice(selectable_feat_2))

                    #Generating a smoothing coefficient for the first feature if necessary
                    #as the first feature is necessarily non-categorical
                    if (new_coef[1][:-1] == "temp_liss"):
                        new_coef.insert(2, np.random.uniform(self.alpha_min, self.alpha_max))

                else:
                    #Ensuring that the new feature has not already been selected in the GAM
                    selectable_features = [x for x in self.feature_list if not x in selected_features]
                    new_coef.append(np.random.choice(selectable_features)) 

                #Generating a smoothing coefficient if necessary
                if (new_coef[-1][:-1] == "temp_liss"):
                    new_coef.append(np.random.uniform(self.alpha_min, self.alpha_max))
                #Generating a list of types if necessary
                elif (new_coef[-1] == "offset"):
                    new_coef.append(self.offset_dynExcBl.random())
                elif (new_coef[-1] == "DayType"):
                    new_coef.append(self.daytype_dynExcBl.random())                      

                if (new_coef[0] in self.categorical_list):
                    new_coef.append("linear")
                    new_coef.append(0)
                else:
                    new_coef.append(np.random.choice(self.spline_list))
                    new_coef.append(np.random.randint(self.k_min, self.k_max+1))
                    #These splines only work in R with k=4 at least
                    if (new_coef[-2] in ['ds', 'cc', 'ps']):
                        new_coef[-1] = max(new_coef[-1], 4)
                    #This spline only works in R with k=5 at least
                    elif (new_coef[-2] == 'bs'):
                        new_coef[-1] = max(new_coef[-1], 5)
                
                gam.append(new_coef)

            res.append(gam)
    
        if (size == 1):
            return res[0]
        else:
            return res

    def isconstant(self):
        return False
    

class ExclusiveBlockInterval(VarNeighborhood):
    def __init__(self, neighborhood=None, variable=None):
        self.neighborhood = neighborhood
        super(ExclusiveBlockInterval, self).__init__(variable)

    def __call__(self, value, size=1):
        res = []
        for _ in size:
            inter = copy.deepcopy(value)
            variables_idx = sorted(list(set(np.random.choice(range(self.target.repeat), size=self.target.repeat))))
            for i in variables_idx:
                inter[i] = self.target.value.neighbor(value[i])
                
                #Verification of the unicity of each value
                while (inter[i] in inter[:i]):
                    inter[i] = self.target.value.neighbor(inter[i])

                res.append(inter)

        if (size == 1):
            return res[0]
        else:
            return res

    @VarNeighborhood.neighborhood.setter
    def neighborhood(self, neighborhood=None):
            self._neighborhood = neighborhood

    @VarNeighborhood.target.setter
    def target(self, variable):
        assert(
            isinstance(variable, ExclusiveBlock) or variable is None
        ), f"""Target object must be a `Block` for {self.__class__.__name__},\
        got {variable}."""
        
        self._target = variable

        if variable is not None:
            assert(
                hasattr(self._target.value, "neighbor")
            ),  f"""To use `ExclusiveBlock`, value for `ExclusiveBlock` must have a `neighbor` method. Use `neighbor` kwarg when defining a variable."""


class DynamicExclusiveBlockInterval(VarNeighborhood):
    def __call__(self, value, size=1, new_repeat=None):
        res = []
        for _ in range(size):
            inter = copy.deepcopy(value)

            if new_repeat is None:
                #If a new length for value has not explicitly been given,
                #we choose a new one as a function of the given neighborhood
                #while remaining in the given length limits
                new_repeat = np.random.randint(max(len(inter) - self._neighborhood, self.target.min_repeat),
                                                min(len(inter) + self._neighborhood, self.target.repeat) + 1)

            #Adding new unique coefficients if the new length is greater
            if new_repeat > len(inter):
                diff = new_repeat - len(inter)
                for _ in range(diff):
                    add_coef = self.target.value.random()
                    while add_coef in inter:
                        add_coef = self.target.value.random()
                    inter.append(add_coef)

            #Removing values at random if the new length is smaller
            if new_repeat < len(inter):
                deleted_idx = np.random.choice(range(len(inter)), size = len(inter)-new_repeat, replace=False)
                for index in sorted(deleted_idx, reverse=True):
                    del inter[index]

            #The following line chooses value indices to be mutated
            variables_idx = sorted(list(set(np.random.choice(range(new_repeat), size=new_repeat))))

            for i in variables_idx:
                inter[i] = self.target.value.neighbor(inter[i])

                #Verification of the unicity of each value
                while (inter[i] in inter[:i]):
                    inter[i] = self.target.value.neighbor(inter[i])

            res.append(inter)

        if size == 1:
            return res[0]
        else:
            return res

    @VarNeighborhood.neighborhood.setter
    def neighborhood(self, neighborhood=None):
        if isinstance(neighborhood, list):
            self._neighborhood = neighborhood[0]
            self.target.value.neighborhood = neighborhood[1]
        else:
            self._neighborhood = neighborhood

    @VarNeighborhood.target.setter
    def target(self, variable):
        assert(
            isinstance(variable, DynamicExclusiveBlock) or variable is None
        ), f"""Target object must be a `DynamicExclusiveBlock` for {self.__class__.__name__}, got {variable}."""

        self._target = variable

        if variable is not None:
            assert(
                hasattr(self._target.value, "neighbor")
            ), f"""To use `DynamicExclusiveBlock`, value for `DynamicExclusiveBlock` must have a `neighbor` method. Use `neighbor` kwarg when defining a variable."""


class GamCoefInterval(VarNeighborhood):
    """
    Neighborhood funtion to mutatea GamCoef variable

    Parameters
    ----------
    k_interval     : Degree of freedom variation amplitude for a mutation (new_k = current_k ± k_interval with new_k automatically staying within its bounds)
    alpha_interval : Smoothing coefficient variation amplitude for a mutation (new_alpha = current_alpha ± alpha_interval with new_alpha automatically staying within its bounds)
    daytype_interval and offset_interval :
            Amplitude of variation of the selected coefficients list's length
            Example :
            If there are 8 selected coefficients and the amplitude is 3, then there can be between 8-3=5 to 8+3=11 selected coefficients after mutation.
            The number of selected coefficients is automatically clipped between 2 and all the available coefficients.
   """
    def __init__(self, k_interval=5, alpha_interval=0.03, offset_interval=5, daytype_interval=5, neighborhood=None, variable=None):
        super(GamCoefInterval, self).__init__(variable)
        self.k_interval = k_interval
        self.alpha_interval = alpha_interval
        self.offset_interval = offset_interval
        self.daytype_interval = daytype_interval
        self._neighborhood = neighborhood
        print("Interval: ", self.k_interval, " ", k_interval)


    def __call__(self, value, size=1):
        features = CatVar("features", self.target.feature_list, neighbor=CatInterval())
        splines = CatVar("splines", self.target.spline_list, neighbor=CatInterval())
        print(self.k_interval)
        k = IntVar("k", self.target.k_min, self.target.k_max, neighbor=IntInterval(self.k_interval))
        alpha = FloatVar("alpha", self.target.alpha_min, self.target.alpha_max, neighbor=FloatInterval(self.alpha_interval))
        daytype_dynExcBl = DynamicExclusiveBlock("daytype_dynExcBl", 
                                                 CatVar("daytype_catvar", self.target.daytype_list, neighbor=CatInterval()), min_repeat=2, max_repeat=len(self.target.daytype_list), 
                                                 neighbor=DynamicExclusiveBlockInterval(self.daytype_interval))
        offset_dynExcBl = DynamicExclusiveBlock("offset_dynExcBl", CatVar("offset_catvar", self.target.offset_list, neighbor=CatInterval()), min_repeat=2, max_repeat=len(self.target.offset_list), neighbor=DynamicExclusiveBlockInterval(self.offset_interval))

        res = []
        for _ in range(size):
            mut_coef = copy.deepcopy(value)
            selected_idx = sorted(list(set(np.random.choice(range(len(value)), size=len(value)))))

            for i in selected_idx:

                if (i == 0):
                    #Mutating the feature
                    former_coef = mut_coef[0]
                    mut_coef[0] = features.neighbor(mut_coef[0])

                    #Specific to EDF R39: adaptating the additional component when necessary
                    #and removing the need to mutate it
                    if (former_coef[:-1] == "temp_liss"):
                        if (mut_coef[0] == "offset"):
                            mut_coef[1] = offset_dynExcBl.random()
                            if 1 in selected_idx: selected_idx.remove(1)

                        elif (mut_coef[0] == "DayType"):
                            mut_coef[1] = daytype_dynExcBl.random()
                            if 1 in selected_idx: selected_idx.remove(1)

                    elif (former_coef == "offset"):
                        if (mut_coef[0] == "DayType"):
                            mut_coef[1] = daytype_dynExcBl.random()
                            if 1 in selected_idx: selected_idx.remove(1)

                        elif (mut_coef[0][:-1] == "temp_liss"):
                            mut_coef[1] = alpha.random()
                            if 1 in selected_idx: selected_idx.remove(1)

                    elif (former_coef == "DayType"):
                        if (mut_coef[0] == "offset"):
                            mut_coef[1] = offset_dynExcBl.random()
                            if 1 in selected_idx: selected_idx.remove(1)
                            
                        elif (mut_coef[0][:-1] == "temp_liss"):
                            mut_coef[1] = alpha.random()
                            if 1 in selected_idx: selected_idx.remove(1)

                if (i == 1):
                    if (len(value) == 3):
                        #Mutating the spline
                        mut_coef[1] = splines.neighbor(mut_coef[1])
                    else:
                        #Specific to EDF R39
                        #Mutating the additional components
                        if (value[0][:-1] == "temp_liss"):
                            mut_coef[1] = alpha.neighbor(mut_coef[1])
                        elif (value[0] == "DayType"):
                            mut_coef[1] = daytype_dynExcBl.neighbor(mut_coef[1])
                        elif (value[0] == "offset"):
                            mut_coef[1] = offset_dynExcBl.neighbor(mut_coef[1])

                if (i == 2):
                    if (len(value) == 3 and mut_coef[1] != "linear"):
                        #Mutating the degree of freedom
                        #No need to mutate the degree of freedom if the spline is linear
                        mut_coef[2] = k.neighbor(mut_coef[2])
                    elif (len(value) == 4):
                        #Mutating the spline
                        mut_coef[2] = splines.neighbor(mut_coef[2])

                if (i == 3 and mut_coef[2] != "linear"):
                    #Mutating the degree of freedom when the feature is "temp_lissx", "DayType", or "offset"
                    #No need to mutate the degree of freedom if the spline is linear
                    mut_coef[3] = k.neighbor(mut_coef[3])
                
            #These splines only work with k=4 at least
            if (mut_coef[-2] in ['ds', 'cc', 'ps']):
                mut_coef[-1] = max(mut_coef[-1], 4)
            
            #This spline only works with k=5 at least
            elif (mut_coef[-2] == 'bs'):
                mut_coef[-1] = max(mut_coef[-1], 5)

            #Removing the smoothing coefficient, or the daytype/offset list when not needed
            if (len(mut_coef) == 4 and mut_coef[0] not in ["offset", "DayType"] and mut_coef[0][:-1] != "temp_liss"):
                mut_coef.pop(1)
            #Adding them when needed
            elif (len(mut_coef) == 3):
                if (mut_coef[0][:-1] == "temp_liss"):
                    mut_coef.insert(1, alpha.random())
                elif (mut_coef[0] == "offset"):
                    mut_coef.insert(1, offset_dynExcBl.random())
                elif (mut_coef[0] == "DayType"):
                    mut_coef.insert(1, daytype_dynExcBl.random())

            res.append(mut_coef)
    
        if (size == 1):
            return res[0]
        else:
            return res
        
    
    @VarNeighborhood.neighborhood.setter
    def neighborhood(self, neighborhood=None):
        self._neighborhood = neighborhood

    

    @VarNeighborhood.target.setter
    def target(self, variable):
        assert(
            isinstance(variable, GamCoef) or variable is None
        ), f"""Target object must be a `GamCoef` for {self.__class__.__name__}, got {variable}."""

        self._target = variable


class GAMInterval(VarNeighborhood):
    def __call__(self, value, size=1):
        res = []
        for _ in range(size):
            mut_gam = copy.deepcopy(value)
            #Choosing the GAM coefficients to mutate
            selected_idx = sorted(list(set(np.random.choice(range(len(value)), size=len(value)))))

            for i in selected_idx:
                #Making sure each feature only appears once in the mutated GAM
                if (i == 0):
                    selectable_features = self.target.feature_list
                else:
                    selectable_features = [x for x in self.target.feature_list if not x in mut_gam[:i][0]]
                
                artificial_coef = GamCoef("artificial_coef", selectable_features, self.target.spline_list, self.target.k_min, self.target.k_max, self.target.alpha_min, self.target.alpha_max, self.target.daytype_list, self.target.offset_list, neighbor=GamCoefInterval(self.k_interval, self.alpha_interval, self.offset_interval, self.daytype_interval))
                mut_gam[i] = artificial_coef.neighbor(mut_gam[i])

            res.append(mut_gam)
        
        if (size == 1):
            return res[0]
        else:
            return res
    
    @VarNeighborhood.neighborhood.setter
    def neighborhood(self, k_interval=5, alpha_interval=0.03, offset_interval=5, daytype_interval=5, neighborhood=None):
        self.k_interval = k_interval
        self.alpha_interval = alpha_interval
        self.offset_interval = offset_interval
        self.daytype_interval = daytype_interval
        self._neighborhood = neighborhood
    
    @VarNeighborhood.target.setter
    def target(self, variable):
        assert(
            isinstance(variable, GAM) or variable is None
        ), f"""Target object must be a `GAM` for {self.__class__.__name__}, got {variable}."""

        self._target = variable

In [30]:
##"Tendance", "Annee", "isoAnnee", "temp_liss_95", "temp_liss_96", "temp_liss_97", "temp_liss_98", "temp_liss_99", "temp_liss_993", "temp_liss_997", "temp_liss_999"
#"temp_lissx" has a special format since it needs a smoothing coefficient as well
#"temp_lissx" should always be written with a number at its end (where x is), to ensure correct processing in the R code. This is to account for possible multiple occurrences of this variable
#DayType and offset also have special formats
feature_list = ["temp_liss1", "temp_liss2", "temp_liss3", "temp_liss4", "Mois", "Jour", "Posan", "JourSemaine", "JourFerie", "Ponts", "isoSemaine", "offset", "DayType", "Weekend", "temp", "nebu", "wind", "temp_s", "nebu_s", "wind_s", "tempMax", "tempMin", "windMax", "windMin", "nebuMax"]
categorical_list = ["Mois", "Jour", "JourSemaine", "JourFerie", "Ponts", "isoSemaine", "offset", "DayType", "Weekend"]
#"linear" means that the variable will enter parametrically
#All categorical variables enter parametrically, regardless of their spline and k values
spline_list = ["linear", "tp", "ts", "ds", "cr", "cs", "cc", "bs" ,"ps", "cp", "gp"]

#Specific to EDF R39
#daytypes 471, 401 only appear in a day in 2022
daytype_list = [0, 1, 2, 3, 4, 5, 6, 10, 11, 12, 13, 14, 20, 21, 22, 23, 30, 31, 32, 33, 40, 402, 41, 42, 43, 44, 45, 46, 47, 472, 50, 51, 52, 60, 61]
offset_list = [0, 1, 2, 3, 4, 5, 6, 10, 11, 12, 13, 14, 15, 20, 21, 211, 22, 23]

test = GamCoef("gam", feature_list, categorical_list, spline_list, bivariate=True, daytype_list=daytype_list, offset_list=offset_list, neighbor=GamCoefInterval())
a = test.random()
print(a)
print(test.neighbor(a))

Interval:  5   5
[np.str_('te'), np.str_('nebuMax'), np.str_('offset'), [1, 23, 2, 6, 11, 5], np.str_('ds'), 24]
5
[np.str_('te'), np.str_('nebuMax'), np.str_('offset'), [1, 23, 2, 6, 11, 5], np.str_('ds'), 24]


In [6]:
test.neighbor.k_interval